# Offense Code + Description Clean Up
The purpose of this script is to manupulate the NC Offense Codes dataset to capture codes that may have different classes in a separate column

In [2]:
import numpy as np
import pandas as pd

# Merge with offence code descriptions:
offense_desc = pd.read_csv(
    "/Users/clarissaache/Documents/Capstone/JUSTFAIR_capstone/00_raw_data/ACIS_offence_codes.csv",
    low_memory=False,
).reset_index()

# Sentencing grid
sent_grid = pd.read_csv(
    "/Users/clarissaache/Documents/Capstone/JUSTFAIR_capstone/10_clean_data/punishment_grid_db.csv",
    low_memory=False,
)

# Drop columns I dont need
offense_desc.sample(5)
offense_desc = offense_desc[
    ["CODE", "Offense Description", "T", "NC General Statute", "CL"]
]

offense_desc.loc[offense_desc["CODE"].duplicated()]

# rename cols
# offense_desc.rename(columns={"CODE":"conv_CODE", "Offense Description":'conv_Off_Desc','T':'convc_type', "NC General Statute":'conv_statute', "CL":'conv_off_class'}, inplace=True)


# make min and max class columns
df = offense_desc[["CODE", "CL"]]
first = df.loc[~df.duplicated()]
first = first.rename(columns={"CL": "CL_min"})
second = df.groupby(by="CODE").nth(1).reset_index()
second = second.rename(columns={"CL": "CL_max"})
all_clases = pd.merge(first, second, on="CODE", how="left")
all_clases.loc[all_clases["CODE"].isna(), "CL_max"] = all_clases.loc[
    all_clases["CODE"].isna(), "CL_min"
]
all_clases["CL_max"].fillna(all_clases["CL_min"], inplace=True)
all_clases = all_clases.loc[~all_clases["CODE"].duplicated()]

# merge with original descriptions
offense_desc_fixed = pd.merge(offense_desc, all_clases, how="left", on="CODE")

# create ranks for classes
class_ranks = {
    "0": 0,
    "3": 1,
    "2": 2,
    "1": 3,
    "A1": 4,
    "I": 5,
    "H": 6,
    "G": 7,
    "F": 8,
    "E": 9,
    "D": 10,
    "C": 11,
    "B2": 12,
    "B1": 13,
    "A": 14,
    "??": np.nan,
}
class_ranks = pd.DataFrame.from_dict(class_ranks, "index").reset_index()
class_ranks.rename(columns={0: "CL_Rank"}, inplace=True)

# drop duplicates
offense_desc_fixed = offense_desc_fixed[
    ["CODE", "Offense Description", "T", "NC General Statute", "CL_min", "CL_max"]
].drop_duplicates()

# merge with index and rename
offense_desc_fixed2 = pd.merge(
    offense_desc_fixed, class_ranks, left_on="CL_min", right_on="index", how="left"
)
offense_desc_fixed2 = pd.merge(
    offense_desc_fixed2, class_ranks, left_on="CL_max", right_on="index", how="left"
)
offense_desc_fixed2.rename(
    columns={"CL_Rank_x": "CL_Rank_min", "CL_Rank_y": "CL_Rank_max"}, inplace=True
)
offense_desc_final = offense_desc_fixed2[
    [
        "CODE",
        "Offense Description",
        "T",
        "NC General Statute",
        "CL_min",
        "CL_Rank_min",
        "CL_max",
        "CL_Rank_max",
    ]
]

# save
offense_desc_final.to_csv(
    "/Users/clarissaache/Documents/Capstone/JUSTFAIR_capstone/00_raw_data/ACIS_offence_codes_FIXED.csv"
)

# Beautiful!
print(all_clases["CODE"].is_unique)
offense_desc_final.loc[offense_desc_fixed2.CODE == 920]

True


,CODE,Offense Description,T,NC General Statute,CL_min,CL_Rank_min,CL_max,CL_Rank_max
502,920,VOLUNTARY MANSLAUGHTER,F,14-18,E,9.0,D,10.0
